In [2]:
import pandas as pd
import sqlite3
import os

# --- 設定 ---
DB_PATH = "../data/processed/s2orc_filtered.db"

# --- データベースの基本情報を表示 ---
def display_basic_info():
    print("--- Database General Information ---")
    if not os.path.exists(DB_PATH):
        print(f"❌ Error: Database file not found at {DB_PATH}")
        return

    # ファイルサイズの確認 (GB単位)
    db_size_gb = os.path.getsize(DB_PATH) / (1024**3)
    print(f"Database file size: {db_size_gb:.2f} GB")

    with sqlite3.connect(DB_PATH) as conn:
        # テーブル一覧の取得
        print("\n--- Tables in Database ---")
        tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
        display(tables)

        # 各テーブルのスキーマ（構造）を表示
        for table_name in tables['name']:
            print(f"\n--- Schema for `{table_name}` table ---")
            schema = pd.read_sql_query(f"PRAGMA table_info({table_name});", conn)
            display(schema)

display_basic_info()

--- Database General Information ---
Database file size: 28.95 GB

--- Tables in Database ---


,name
0,papers
1,citations



--- Schema for `papers` table ---


,cid,name,type,notnull,dflt_value,pk
0,0,corpus_id,INTEGER,0,None,1
1,1,doi,TEXT,0,None,0
2,2,title,TEXT,0,None,0
3,3,abstract,TEXT,0,None,0



--- Schema for `citations` table ---


,cid,name,type,notnull,dflt_value,pk
0,0,citing_doi,TEXT,0,None,0
1,1,cited_doi,TEXT,0,None,0


In [3]:
def analyze_papers_table():
    print("\n" + "="*50)
    print("--- Analysis of `papers` Table ---")
    if not os.path.exists(DB_PATH): return

    with sqlite3.connect(DB_PATH) as conn:
        # 総論文数
        total_count = pd.read_sql_query("SELECT COUNT(*) FROM papers", conn).iloc[0, 0]
        print(f"Total rows (papers): {total_count:,}")

        # ユニークなDOIの数（総論文数と一致するはず）
        unique_doi_count = pd.read_sql_query("SELECT COUNT(DISTINCT doi) FROM papers", conn).iloc[0, 0]
        print(f"Unique DOIs: {unique_doi_count:,}")
        if total_count != unique_doi_count:
            print("⚠️ Warning: Row count and unique DOI count do not match.")

        # タイトルが欠損している論文数
        missing_title_count = pd.read_sql_query("SELECT COUNT(*) FROM papers WHERE title IS NULL OR title = ''", conn).iloc[0, 0]
        print(f"Papers with missing title: {missing_title_count:,}")
        
        # アブストラクトが欠損している論文数
        missing_abstract_count = pd.read_sql_query("SELECT COUNT(*) FROM papers WHERE abstract IS NULL OR abstract = ''", conn).iloc[0, 0]
        print(f"Papers with missing abstract: {missing_abstract_count:,}")

analyze_papers_table()


--- Analysis of `papers` Table ---
Total rows (papers): 11,619,136
Unique DOIs: 11,619,136
Papers with missing title: 446,635
Papers with missing abstract: 0


In [4]:
def analyze_citations_table():
    print("\n" + "="*50)
    print("--- Analysis of `citations` Table ---")
    if not os.path.exists(DB_PATH): return

    with sqlite3.connect(DB_PATH) as conn:
        # 総引用リンク数
        total_citations = pd.read_sql_query("SELECT COUNT(*) FROM citations", conn).iloc[0, 0]
        print(f"Total citation links: {total_citations:,}")

        # 最も多く引用されている論文トップ10
        print("\n--- Top 10 Most Cited Papers (within this dataset) ---")
        query = """
            SELECT 
                T2.title,
                T1.cited_doi, 
                COUNT(T1.cited_doi) AS citation_count
            FROM citations AS T1
            JOIN papers AS T2 ON T1.cited_doi = T2.doi
            GROUP BY T1.cited_doi
            ORDER BY citation_count DESC
            LIMIT 10;
        """
        top_cited = pd.read_sql_query(query, conn)
        display(top_cited)
        
        # 最も多くの文献を引用している論文トップ10
        print("\n--- Top 10 Papers Citing the Most (Longest Reference Lists) ---")
        query = """
            SELECT 
                T2.title,
                T1.citing_doi, 
                COUNT(T1.citing_doi) AS reference_count
            FROM citations AS T1
            JOIN papers AS T2 ON T1.citing_doi = T2.doi
            GROUP BY T1.citing_doi
            ORDER BY reference_count DESC
            LIMIT 10;
        """
        top_citing = pd.read_sql_query(query, conn)
        display(top_citing)

analyze_citations_table()


--- Analysis of `citations` Table ---
Total citation links: 91,393,156

--- Top 10 Most Cited Papers (within this dataset) ---


,title,cited_doi,citation_count
0,"Scientific Vice President, Surveillance and He...",10.3322/CAAC.21492,12984
1,,10.3322/CAAC.21660,12360
2,Genome analysis Trimmomatic: a flexible trimme...,10.1093/BIOINFORMATICS/BTU170,8250
3,Fitting linear mixed-effects models using lme4,10.18637/JSS.V067.I01,7598
4,Clinical features of patients infected with 20...,10.1016/S0140-6736(20)30183-5,6987
5,,10.1017/CBO9781107415324.004,6418
6,The Sequence Alignment/Map format and SAMtools,10.1093/BIOINFORMATICS/BTP352,4896
7,BERT: Pre-training of Deep Bidirectional Trans...,10.18653/V1/N19-1423,4199
8,Strelka: accurate somatic small-variant callin...,10.1093/BIOINFORMATICS/BTP324,3913
9,Article Fast Track MAFFT Multiple Sequence Ali...,10.1093/MOLBEV/MST010,3840



--- Top 10 Papers Citing the Most (Longest Reference Lists) ---


,title,citing_doi,reference_count
0,Supplementary file: Ethical reporting of resea...,10.1136/BMJGH-2023-011882,9195
1,Among Physicians in Nepal,10.55729/2000-9666.1101,8080
2,Three-Port Bi-Directional DC-DC Converter with...,10.3390/EN16020624,7896
3,Coronary Artery Dissection and Myocarditis Cau...,10.55729/2000-9666.1219,4550
4,Photocrosslinking Probes Proximity of Thymine ...,10.3390/MOLECULES27134006,4109
5,Synergism of calycosin and bone marrow-derived...,10.4252/WJSC.V15.I6.617,3968
6,Nachhaltigkeit in der chirurgischen Niederlass...,10.1007/S00104-022-01785-7,3876
7,REVIEW Open Access Continuous renal replacemen...,10.1186/S13054-023-04555-X,3801
8,Strategies to manage the difficult colonoscopy,10.4253/WJGE.V15.I7.491,3500
9,Why We Belong -Exploring Membership of Healthc...,10.2196/RESPROT.5323,3456
